In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings

In [2]:
warnings.filterwarnings("ignore")
jobOrder_fam = pd.read_csv("jobOrder_Clean.csv", sep='|', encoding='latin-1')

In [3]:
jobOrder_fam["PRIMARY_KEY"] = jobOrder_fam["CONF_CODE"].astype(str) + "_" + jobOrder_fam["ORDER_ROW"].astype(str) + "_" + jobOrder_fam["JO_PROG"].astype(str)

In [4]:
jobOrder_fam["LAUNCH_DATE"] = pd.to_datetime(jobOrder_fam["LAUNCH_DATE"])

In [5]:
jobOrder_fam = jobOrder_fam[jobOrder_fam['SCHEDULED_QTY'] == 1]

In [470]:
not_relevant = ["SCHEDULED_QTY"]
jobOrder_fam.drop(columns=not_relevant, inplace=True)

In [478]:
len(jobOrder_fam["PRODUCT_CODE"].unique())

817

In [479]:
category_material = jobOrder_fam["MATERIAL_CODE"].value_counts()

3201.0    5009149
6401.0    2113676
7401.0    1691578
1001.0    1132151
6001.0     948030
2301.0     526170
2801.0     377653
401.0      314757
6101.0      42702
4101.0      41229
6201.0      22965
8001.0       3198
3001.0       2126
6301.0       1568
2001.0       1261
9901.0       1116
2201.0        718
7001.0        358
201.0         335
7101.0        330
5501.0        253
6901.0        238
7301.0         72
7201.0         42
1301.0         32
6110.0         18
8101.0         15
801.0          14
210.0           6
1511.0          5
6501.0          2
2101.0          1
Name: MATERIAL_CODE, dtype: int64

In [480]:
ranks = category_material.rank(ascending=False, method='min')

In [482]:
jobOrder_fam['category_material'] = jobOrder_fam['MATERIAL_CODE'].map(ranks)

In [503]:
jobOrder_new = jobOrder_fam[jobOrder_fam['LAUNCH_DATE'].notnull()]

In [504]:
warnings.filterwarnings("ignore")
jobOrder_new['month'] = jobOrder_new['LAUNCH_DATE'].dt.month
warnings.filterwarnings("default")

In [506]:
null_counts = jobOrder_new.isnull().sum()
print(null_counts)

CONF_CODE                  0
ORDER_ROW                  0
JO_PROG                    0
CUST_CODE                  0
PRIORITY_COMP              0
PRODUCT_CODE               0
MATERIAL_CODE              0
BATCH_NUMBER               0
LATHE_SETUP               12
BASE_RADIUS                0
POWER                      0
CYLINDER             2766320
POWER_AXIS           2766320
DIAMETER                   0
CENTER_THK                 0
SCHEDULED_DATE             0
LAUNCH_DATE                0
REJECTS_QTY                0
DELIVERY_DATE         144739
Adicion              8952112
ItemCode              120963
ItemName                 156
LinProd                  156
CodFam                   156
CodSubFam                156
PRIMARY_KEY                0
category_material          0
month                      0
dtype: int64


In [508]:
category_fam = jobOrder_new["CodFam"].value_counts()

1.0     10858985
2.0       811883
3.0       268321
10.0      120963
14.0      111298
13.0           2
Name: CodFam, dtype: int64

In [509]:
ranks = category_fam.rank(ascending=False, method='min')

1.0     1.0
2.0     2.0
3.0     3.0
10.0    4.0
14.0    5.0
13.0    6.0
Name: CodFam, dtype: float64

In [510]:
jobOrder_new['category_fam'] = jobOrder_new['CodFam'].map(ranks)

/var/folders/6x/6k8btdy572s70bshk282zpvr0000gn/T/ipykernel_58436/2695955486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobOrder_new['category_fam'] = jobOrder_new['CodFam'].map(ranks)


In [512]:
category_subfam = jobOrder_new["CodSubFam"].value_counts()

3.0    6274927
6.0    3073618
1.0    1368694
5.0    1333249
8.0     120963
2.0          1
Name: CodSubFam, dtype: int64

In [513]:
ranks = category_subfam.rank(ascending=False, method='min')
jobOrder_new['category_subfam'] = jobOrder_new['CodSubFam'].map(ranks)

/var/folders/6x/6k8btdy572s70bshk282zpvr0000gn/T/ipykernel_58436/3421266748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobOrder_new['category_subfam'] = jobOrder_new['CodSubFam'].map(ranks)


In [515]:
jobOrder_new['category_subfam'].value_counts()

1.0    6274927
2.0    3073618
3.0    1368694
4.0    1333249
5.0     120963
6.0          1
Name: category_subfam, dtype: int64

As you can observe in the value counts above, there are some values that are only repeated a few times, therefore, for the following categories, based on their distribution, we decided on different thresholds to remove some data.

In [516]:
code_counts = jobOrder_new['PRODUCT_CODE'].value_counts()
codes_to_drop = code_counts[code_counts < 5000].index
jobOrder_new = jobOrder_new[~jobOrder_new['PRODUCT_CODE'].isin(codes_to_drop)]

In [517]:
code_counts = jobOrder_new['MATERIAL_CODE'].value_counts()
codes_to_drop = code_counts[code_counts < 3500].index
jobOrder_new = jobOrder_new[~jobOrder_new['MATERIAL_CODE'].isin(codes_to_drop)]

In [518]:
code_counts = jobOrder_new['BASE_RADIUS'].value_counts()
codes_to_drop = code_counts[code_counts < 2300].index
jobOrder_new = jobOrder_new[~jobOrder_new['BASE_RADIUS'].isin(codes_to_drop)]

In [519]:
code_counts = jobOrder_new['POWER'].value_counts()
codes_to_drop = code_counts[code_counts < 3500].index
jobOrder_new = jobOrder_new[~jobOrder_new['POWER'].isin(codes_to_drop)]

In [520]:
code_counts = jobOrder_new['CYLINDER'].value_counts()
codes_to_drop = code_counts[code_counts < 2700].index
jobOrder_new = jobOrder_new[~jobOrder_new['CYLINDER'].isin(codes_to_drop)]

In [521]:
code_counts = jobOrder_new['POWER_AXIS'].value_counts()
codes_to_drop = code_counts[code_counts < 19300].index
jobOrder_new = jobOrder_new[~jobOrder_new['POWER_AXIS'].isin(codes_to_drop)]

In [522]:
code_counts = jobOrder_new['DIAMETER'].value_counts()
codes_to_drop = code_counts[code_counts < 900].index
jobOrder_new = jobOrder_new[~jobOrder_new['DIAMETER'].isin(codes_to_drop)]

In [523]:
code_counts = jobOrder_new['CENTER_THK'].value_counts()
codes_to_drop = code_counts[code_counts < 1000].index
jobOrder_new = jobOrder_new[~jobOrder_new['CENTER_THK'].isin(codes_to_drop)]

In [524]:
code_counts = jobOrder_new['category_fam'].value_counts()
codes_to_drop = code_counts[code_counts < 2].index
jobOrder_new = jobOrder_new[~jobOrder_new['category_fam'].isin(codes_to_drop)]

In [525]:
code_counts = jobOrder_new['category_subfam'].value_counts()
codes_to_drop = code_counts[code_counts < 1].index
jobOrder_new = jobOrder_new[~jobOrder_new['category_subfam'].isin(codes_to_drop)]

In [526]:
null_counts = jobOrder_new.isnull().sum()

print(null_counts)

CONF_CODE                  0
ORDER_ROW                  0
JO_PROG                    0
CUST_CODE                  0
PRIORITY_COMP              0
PRODUCT_CODE               0
MATERIAL_CODE              0
BATCH_NUMBER               0
LATHE_SETUP                0
BASE_RADIUS                0
POWER                      0
CYLINDER             2545395
POWER_AXIS           2545395
DIAMETER                   0
CENTER_THK                 0
SCHEDULED_DATE             0
LAUNCH_DATE                0
REJECTS_QTY                0
DELIVERY_DATE          74118
Adicion              7889241
ItemCode               50113
ItemName                 150
LinProd                  150
CodFam                   150
CodSubFam                150
PRIMARY_KEY                0
category_material          0
month                      0
category_fam             150
category_subfam          150
dtype: int64


In [527]:
jobOrder_cluster = jobOrder_new[jobOrder_new['LAUNCH_DATE'].dt.year >= 2020]

In [528]:
mask = jobOrder_cluster['DELIVERY_DATE'].notnull()

jobOrder_cluster = jobOrder_cluster[mask]

In [529]:
mask = jobOrder_cluster['ItemCode'].notnull() & jobOrder_cluster['ItemName'].notnull() & jobOrder_cluster['LinProd'].notnull() & jobOrder_cluster['CodFam'].notnull() & jobOrder_cluster['CodSubFam'].notnull()

In [530]:
null_counts = jobOrder_cluster.isnull().sum()

print(null_counts)

CONF_CODE                  0
ORDER_ROW                  0
JO_PROG                    0
CUST_CODE                  0
PRIORITY_COMP              0
PRODUCT_CODE               0
MATERIAL_CODE              0
BATCH_NUMBER               0
LATHE_SETUP                0
BASE_RADIUS                0
POWER                      0
CYLINDER             1183827
POWER_AXIS           1183827
DIAMETER                   0
CENTER_THK                 0
SCHEDULED_DATE             0
LAUNCH_DATE                0
REJECTS_QTY                0
DELIVERY_DATE              0
Adicion              3166420
ItemCode                 275
ItemName                  59
LinProd                   59
CodFam                    59
CodSubFam                 59
PRIMARY_KEY                0
category_material          0
month                      0
category_fam              59
category_subfam           59
dtype: int64


In [531]:
jobOrder_cluster = jobOrder_cluster.dropna(subset=['category_fam', 'category_subfam'])

In [532]:
mask = jobOrder_cluster['CYLINDER'].isnull() & jobOrder_cluster['POWER_AXIS'].isnull()

jobOrder_A = jobOrder_cluster[mask]

(1183827, 30)

In [533]:
null_counts = jobOrder_A.isnull().sum()
print(null_counts)

CONF_CODE                  0
ORDER_ROW                  0
JO_PROG                    0
CUST_CODE                  0
PRIORITY_COMP              0
PRODUCT_CODE               0
MATERIAL_CODE              0
BATCH_NUMBER               0
LATHE_SETUP                0
BASE_RADIUS                0
POWER                      0
CYLINDER             1183827
POWER_AXIS           1183827
DIAMETER                   0
CENTER_THK                 0
SCHEDULED_DATE             0
LAUNCH_DATE                0
REJECTS_QTY                0
DELIVERY_DATE              0
Adicion               892224
ItemCode                 135
ItemName                   0
LinProd                    0
CodFam                     0
CodSubFam                  0
PRIMARY_KEY                0
category_material          0
month                      0
category_fam               0
category_subfam            0
dtype: int64


In [534]:
mask = jobOrder_cluster['CYLINDER'].notnull() & jobOrder_cluster['POWER_AXIS'].notnull()

jobOrder_B = jobOrder_cluster[mask]

(2928071, 30)

In [535]:
null_counts = jobOrder_B.isnull().sum()
print(null_counts)

CONF_CODE                  0
ORDER_ROW                  0
JO_PROG                    0
CUST_CODE                  0
PRIORITY_COMP              0
PRODUCT_CODE               0
MATERIAL_CODE              0
BATCH_NUMBER               0
LATHE_SETUP                0
BASE_RADIUS                0
POWER                      0
CYLINDER                   0
POWER_AXIS                 0
DIAMETER                   0
CENTER_THK                 0
SCHEDULED_DATE             0
LAUNCH_DATE                0
REJECTS_QTY                0
DELIVERY_DATE              0
Adicion              2274137
ItemCode                 140
ItemName                   0
LinProd                    0
CodFam                     0
CodSubFam                  0
PRIMARY_KEY                0
category_material          0
month                      0
category_fam               0
category_subfam            0
dtype: int64


In [536]:
jobOrder_A.shape[0] + jobOrder_B.shape[0] == jobOrder_cluster.shape[0]

True

In [537]:
warnings.filterwarnings("ignore")
not_relevant = ["CONF_CODE", "CUST_CODE", "SCHEDULED_DATE", "DELIVERY_DATE", "ORDER_ROW", "JO_PROG", "PRODUCT_CODE",
                "BATCH_NUMBER", "LATHE_SETUP", "REJECTS_QTY", "PRIORITY_COMP", "Adicion", "ItemCode", "ItemName", "LinProd"]
jobOrder_A.drop(columns=not_relevant, inplace=True)
jobOrder_B.drop(columns=not_relevant, inplace=True)
warnings.filterwarnings("default")

In [538]:
jobOrder_new["category_fam"].value_counts()

1.0    9615592
2.0     747395
3.0     245373
5.0     104960
4.0      50113
Name: category_fam, dtype: int64

In [539]:
jobOrder_A["category_fam"].value_counts()

1.0    1126665
2.0      26193
5.0      16952
3.0      13882
4.0        135
Name: category_fam, dtype: int64

In [540]:
jobOrder_B["category_fam"].value_counts()

1.0    2704665
2.0     102080
5.0      87521
3.0      33665
4.0        140
Name: category_fam, dtype: int64

In [541]:
jobOrder_new["category_subfam"].value_counts()

1.0    5454545
2.0    2738593
4.0    1304420
3.0    1215762
5.0      50113
Name: category_subfam, dtype: int64

In [542]:
jobOrder_A["category_subfam"].value_counts()

4.0    737604
3.0    446088
5.0       135
Name: category_subfam, dtype: int64

In [543]:
jobOrder_B["category_subfam"].value_counts()

1.0    1795973
2.0    1131958
5.0        140
Name: category_subfam, dtype: int64

In [544]:
# Organising columns

column_to_move = 'PRIMARY_KEY'

new_order = [column_to_move] + [col for col in jobOrder_A.columns if col != column_to_move]
jobOrder_A = jobOrder_A.reindex(columns=new_order)

new_order = [column_to_move] + [col for col in jobOrder_B.columns if col != column_to_move]
jobOrder_B = jobOrder_B.reindex(columns=new_order)

In [545]:
# Data to cluster
jobOrder_new.to_csv("jobOrder_new.csv", index=False)
jobOrder_B.to_csv("jobOrder_B.csv", index=False)
jobOrder_A.to_csv("jobOrder_A.csv", index=False)